In [1]:
from importlib import reload
import sys
import util
reload(util)
data = util.FontData
FontDataset = util.FontDataset
import experiment
reload(experiment)
from sklearn import linear_model, neural_network

In [2]:
data.load(knn_path="data/ensemble_dataset.csv")

Loading embeddings...done
Loading typographic + semantic vectors...done
Loading images...done
Loading glyphs...done
Loading SVGs...done


In [3]:
train_dataset = FontDataset(data, 'train')
val_dataset = FontDataset(data, 'val')

In [4]:
train_typographic = data.get_all_typographic('train')
val_typographic = data.get_all_typographic('val')

In [5]:
train_image = data.get_all_image('train')
train_image = train_image.reshape(train_image.shape[0], -1)
val_image = data.get_all_image('val')
val_image = val_image.reshape(val_image.shape[0], -1)

In [6]:
train_semantic = data.get_all_semantic('train')
val_semantic = data.get_all_semantic('val')

### Linear Regression: Image -> Typographic

In [7]:
image_typographic_reg = linear_model.LinearRegression()

In [8]:
image_typographic_reg.fit(train_image, train_typographic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [9]:
train_stats, train_typographic_pred = experiment.run_experiment(image_typographic_reg, train_image, train_typographic)

{'RMSE': 1.6118419807753526e-06, 'R2': 0.9999999997904643}


In [10]:
val_stats, val_typographic_pred = experiment.run_experiment(image_typographic_reg, val_image, val_typographic)

{'RMSE': 0.11453434824943542, 'R2': 0.5423802791030774}


### Linear Regression: Typographic -> Semantic

In [11]:
typographic_semantic_reg = linear_model.LinearRegression()

In [12]:
typographic_semantic_reg.fit(train_typographic, train_semantic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [13]:
train_stats, train_semantic_pred = experiment.run_experiment(typographic_semantic_reg, train_typographic, train_semantic)

{'RMSE': 0.1091596782207489, 'R2': 0.428152916505971}


In [14]:
val_stats, val_semantic_pred = experiment.run_experiment(typographic_semantic_reg, val_typographic, val_semantic)

{'RMSE': 0.11040398478507996, 'R2': 0.46110416503091683}


### Linear Regressions: Image -> Typographic -> Semantic

In [15]:
class SequentialLinearRegression:
    def __init__(self, reg1, reg2):
        self.reg1 = reg1
        self.reg2 = reg2
    def predict(self, X):
        return self.reg2.predict(self.reg1.predict(X))

image_typographic_semantic_reg = SequentialLinearRegression(image_typographic_reg, typographic_semantic_reg)

In [16]:
train_semantic

array([[0.33920993, 0.27120551, 0.45607575, ..., 0.48699813, 0.65168096,
        0.54990847],
       [0.49424582, 0.47655215, 0.37099306, ..., 0.68936142, 0.51278904,
        0.48553453],
       [0.24767428, 0.1125422 , 0.25545198, ..., 0.69045158, 0.46161946,
        0.45430069],
       ...,
       [0.73453555, 0.69947555, 0.56249398, ..., 0.49409655, 0.79993149,
        0.45068339],
       [0.74848685, 0.6211679 , 0.45640533, ..., 0.52525637, 0.49403348,
        0.35790375],
       [0.71824549, 0.82481607, 0.47846678, ..., 0.76230265, 0.5292319 ,
        0.31400816]])

In [17]:
train_stats, train_semantic_pred = experiment.run_experiment(image_typographic_semantic_reg, train_image, train_semantic)

{'RMSE': 0.10915966331958771, 'R2': 0.42815299749953795}


In [18]:
val_stats, val_semantic_pred = experiment.run_experiment(image_typographic_semantic_reg, val_image, val_semantic)

{'RMSE': 0.1210595890879631, 'R2': 0.35492018567186}


### Linear Regression: Image -> Semantic

In [19]:
image_semantic_reg = linear_model.LinearRegression()

In [20]:
image_semantic_reg.fit(train_image, train_semantic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [21]:
train_stats, train_semantic_pred = experiment.run_experiment(image_semantic_reg, train_image, train_semantic)

{'RMSE': 1.3222984307503793e-06, 'R2': 0.999999999918881}


In [22]:
val_stats, val_semantic_pred = experiment.run_experiment(image_semantic_reg, val_image, val_semantic)

{'RMSE': 0.07829689234495163, 'R2': 0.7185767623190631}


### Multilayer Perceptron: Image -> Semantic

In [30]:
image_semantic_nn = neural_network.MLPRegressor(hidden_layer_sizes=(10,), max_iter=500)

In [31]:
image_semantic_nn.fit(train_image, train_semantic)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(10,), learning_rate='constant',
             learning_rate_init=0.001, max_iter=500, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [32]:
train_stats, train_semantic_pred = experiment.run_experiment(image_semantic_nn, train_image, train_semantic)

{'RMSE': 0.16700612008571625, 'R2': -0.41217107335012654}


In [33]:
val_stats, val_semantic_pred = experiment.run_experiment(image_semantic_nn, val_image, val_semantic)

{'RMSE': 0.1747315675020218, 'R2': -0.3753219676038256}


#### Example